# Single Map, some other parameter

* Use papermill to inject district parameter
* Within notebook, have it look up some column within the district and inject that column info into header

In [1]:
%%capture
import branca
import calitp.magics
import geopandas as gpd
import pandas as pd

# Alternatively, can import relevant ones
from shared_utils import map_utils

In [2]:
## parameters cell
district = 7

In [3]:
# Parameters
district = 4


In [4]:
GEOSPATIAL_DATA = ("https://caltrans-gis.dot.ca.gov/arcgis/rest/services/"
       "CHboundary/District_Tiger_Lines/FeatureServer/0/query?"
       "where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&"
       "inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&"
       "relationParam=&outFields=DISTRICT%2C+Region&returnGeometry=true&"
       "maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&gdbVersion=&"
       "historicMoment=&returnDistinctValues=false&returnIdsOnly=false&"
       "returnCountOnly=false&returnExtentOnly=false&orderByFields=&"
       "groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&"
       "multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&"
       "returnTrueCurves=false&returnExceededLimitFeatures=false&quantizationParameters=&"
       "returnCentroid=false&sqlFormat=none&resultType=&featureEncoding=esriDefault&"
       "datumTransformation=&f=geojson"
      )

gdf = gpd.read_file(GEOSPATIAL_DATA)

# Create a y-col to map (that's not just District number)
gdf = gdf.assign(
    some_y_col = gdf.index+10
)

gdf = gdf[gdf.DISTRICT==district]

In [5]:
%%capture_parameters
some_metric = str(gdf.some_y_col.iloc[0])
district, some_metric

# District 4 ------ grabbed metric: 13

In [6]:
popup_dict = {
    "DISTRICT": "Caltrans District",
    "Region": "Caltrans Region",
    "some_y_col": "Something Measured"
}

colorscale = (branca.colormap.step.Accent_07
              .scale(vmin=gdf.DISTRICT.min(), 
                     vmax=gdf.DISTRICT.max())
             )

m = map_utils.make_folium_choropleth_map(
    gdf,
    plot_col = "DISTRICT",
    popup_dict = popup_dict,
    tooltip_dict = popup_dict,
    colorscale = colorscale,
    fig_width = 500, fig_height = 800,
    zoom = map_utils.REGION_CENTROIDS["CA"]["zoom"],
    centroid = map_utils.REGION_CENTROIDS["CA"]["centroid"],
    title=f"District {district} Map"
)

m